In [12]:
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
import ssl
import os
import csv


class Daily539:
    def __init__(self):
        self.lottery_list = []

    def get_history_from_page(self, indexpage, prev_date):
        # disable ssl certificate verify
        ssl._create_default_https_context = ssl._create_unverified_context
        url = 'https://www.lotto-8.com/server/lto539/serverD/listlto539bbk.asp?indexpage=' + \
            str(indexpage) + '&orderby=old'
        response = urlopen(url)
        html = response.read()

        soup = BeautifulSoup(html, "html.parser")
        history = soup.find('table', 'auto-style4')
        content = history.find_all('td')

        # 將內容改成3-tuple, data/number/space, 並跳過第一欄說明欄
        content = zip(*[content[3:][i::3] for i in range(3)])
        cur_date = None
        for result in content:
            date = result[0].text
            if cur_date is None:
                cur_date = date
            numbers = (result[1].text).replace(' ', '').split(',')

            result = [0] * 39
            for num in numbers:
                result[int(num)-1] = 1
            data = {"date": date}
            for i in range(39):
                data[str(i+1)] = result[i]
            self.lottery_list.append(data)
        # 持續抓歷史資料，直到抓到重複的為止
        if(cur_date != prev_date):
            self.get_history_from_page(indexpage+1, cur_date)

    def write_history_to_file(self):
        print("\nGet data from website...\n")
        self.get_history_from_page(1, None)

        print("\nWrite history results to local file...")
        header = ["date"]
        for i in range(39):
                header.append(str(i+1))
        with open("data/history.csv", 'w') as output_file:
            dict_writer = csv.DictWriter(output_file, header)
            dict_writer.writeheader()
            dict_writer.writerows(self.lottery_list)

#     def read_numbers_to_file(self):
#         print("\nRead history results from local file...")
#         f = open("lottery_numbers.txt", 'r')
#         for line in f.readlines():
#             line = line.strip().split()
#             numbers = line[1].split(",")
#             numbers = list(map(int, numbers))
#             data = {"date": line[0], "numbers": numbers}
#             self.lottery_list.append(data)
#         print("Finish.\n")
#         print(
#             "*There are total of %d history data.\n" % len(self.lottery_list))

In [13]:
daily539 = Daily539()
daily539.write_history_to_file()


Get data from website...


Write history results to local file...


In [26]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/history.csv')
df = df.drop(['date'], axis=1)
df[:10]

,1,2,3,4,5,6,7,8,9,10,...,30,31,32,33,34,35,36,37,38,39
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,1,0,0,1,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [269]:
def data_helper(df, time_frame):
    
    # 資料維度: 1~39
    number_features = len(df.columns) - 1
    # 將dataframe 轉成 numpy array
    datavalue = df.as_matrix()
    result = []
    # 若想要觀察的 time_frame 為20天, 需要多加一天做為驗證答案
    for index in range( len(datavalue) - (time_frame+1) ): # 從 datavalue 的第0個跑到倒數第 time_frame+1 個
        result.append(datavalue[index: index + (time_frame+1) ]) # 逐筆取出 time_frame+1 個K棒數值做為一筆 instance
    
    result = np.array(result)
    # print(result.shape)
    # (1082, 21, 28)
    number_train = round(0.9 * result.shape[0]) # 取 result 的前90% instance做為訓練資料
    
    # 前面:指全部資料（1082筆
    # 後面:指第1~20筆資料，也就是除了最後一筆
    # [1:] 取除了第一個Label 的 Feature
    x_train = result[:int(number_train), :-1, 1:] # 訓練資料中, 只取每一個 time_frame 中除了最後一筆的所有資料做為feature
    
    # 前面:指全部資料（1082筆
    # 後面代表只拿最後一筆 也就是第21筆
    y_train = result[:int(number_train), -1, 1:] # 訓練資料中, 取每一個 time_frame 中最後一筆資料的最後一個數值(收盤價)做為答案
    # 拿當日的後面三個Label（因為已經把隔天的答案寫入inpu中

    x_test = result[int(number_train):, :-1, 1:]
    y_test = result[int(number_train):, -1, 1:]
    
#     # 將資料組成變好看一點
#     x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], number_features))
#     x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], number_features)) 
    return [x_train, y_train, x_test, y_test]

# 以20天為一區間進行股價預測
X_train, y_train, X_test, y_test = data_helper(df, 36)
print(X_train.shape)
print(X_train.shape)

(237, 36, 39)
(237, 36, 39)


/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [27]:
msk = np.random.rand(len(df)) < 0.9
train_df = df[msk]
test_df = df[~msk]

X_train = np.array(train_df[:-1])
y_train = np.array(train_df[1:])
X_test = np.array(test_df[:-1])
y_test = np.array(test_df[1:])

In [10]:
import os
def save_model():
    try:
        model.save("SaveModel/Prediction_Model.h5")
        print ("Save Model Success!")
    except:
        print ("Save Model Failed!")

In [32]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.layers import TimeDistributed, Flatten
from keras.layers import Conv2D
from keras.layers import Conv1D

def build_model(input_length, input_dim):
    d = 0.25
    model = Sequential()
    model.add(Dense(units = 1600,
                input_dim = 39,
                activation = 'relu'))
#     model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=True))
#     model.add(LSTM(256, input_shape=(input_length, input_dim), return_sequences=True))
    model.add(Dropout(0.5))
#     model.add(TimeDistributed(Dense(1)))
#     model.add(Flatten())
    model.add(Dense(1600, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1600, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(39, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

# 20天、39維
model = build_model( 1, 39 )

In [ ]:
# 一個batch有128個instance，總共跑50個迭代
train_history = model.fit( X_train, y_train, batch_size=4, epochs=100, verbose=1, validation_split=0.1)
# save_model()

Train on 2961 samples, validate on 329 samples
Epoch 1/100
2961/2961 [==============================] - 64s 22ms/step - loss: 0.3977 - acc: 0.8707 - val_loss: 0.3912 - val_acc: 0.8718
Epoch 2/100
2961/2961 [==============================] - 62s 21ms/step - loss: 0.3887 - acc: 0.8718 - val_loss: 0.3883 - val_acc: 0.8718
Epoch 3/100
2961/2961 [==============================] - 64s 22ms/step - loss: 0.3859 - acc: 0.8718 - val_loss: 0.3870 - val_acc: 0.8718
Epoch 4/100
2961/2961 [==============================] - 82s 28ms/step - loss: 0.3846 - acc: 0.8718 - val_loss: 0.3866 - val_acc: 0.8718
Epoch 5/100
2961/2961 [==============================] - 70s 24ms/step - loss: 0.3837 - acc: 0.8718 - val_loss: 0.3886 - val_acc: 0.8718
Epoch 6/100
2961/2961 [==============================] - 64s 22ms/step - loss: 0.3818 - acc: 0.8718 - val_loss: 0.3909 - val_acc: 0.8718
Epoch 7/100
2961/2961 [==============================] - 80s 27ms/step - loss: 0.3804 - acc: 0.8718 - val_loss: 0.3907 - val_acc: 0

In [21]:
pred = model.predict(X_test)
total = 0
earn = 0
for idx, val in enumerate(pred):
    correct = 0
#     print(X_test[idx][-1])
#     print(val)
    top_inds = val.argsort()[::-1][:5]
    print("== Predict ==")
    for i in top_inds:
#         print(val[i])
        print(i+1)
    print("== Answer ==")
    for i, v in enumerate(y_test[idx]):
        if v == 1 and i in top_inds:
            correct+=1
            print (i+1)
    if correct == 2:
        earn += 50
    if correct == 3:
        earn += 300
    if correct == 4:
        earn += 20000
    if correct == 5:
        earn += 8000000
    total += 50
print(earn)
print(total)

== Predict ==
23
19
33
37
35
== Answer ==
23
== Predict ==
22
38
9
27
15
== Answer ==
9
15
== Predict ==
9
5
10
4
21
== Answer ==
== Predict ==
16
17
32
30
12
== Answer ==
== Predict ==
5
11
10
36
28
== Answer ==
36
== Predict ==
14
30
38
12
29
== Answer ==
14
== Predict ==
23
32
18
2
9
== Answer ==
== Predict ==
2
25
3
37
15
== Answer ==
== Predict ==
29
15
22
27
9
== Answer ==
15
== Predict ==
23
15
38
27
29
== Answer ==
== Predict ==
38
16
27
8
14
== Answer ==
8
== Predict ==
37
16
20
25
3
== Answer ==
3
20
25
== Predict ==
20
16
3
10
34
== Answer ==
10
== Predict ==
16
29
38
8
18
== Answer ==
16
== Predict ==
20
34
38
16
1
== Answer ==
1
== Predict ==
29
13
18
39
3
== Answer ==
== Predict ==
23
10
36
22
1
== Answer ==
23
== Predict ==
22
23
3
36
19
== Answer ==
19
== Predict ==
25
33
16
34
23
== Answer ==
== Predict ==
15
3
30
11
21
== Answer ==
21
== Predict ==
5
29
9
18
8
== Answer ==
8
== Predict ==
34
38
8
29
17
== Answer ==
17
== Predict ==
32
29
26
38
9
== Answer ==
29
== Pre

31
== Predict ==
27
38
9
26
23
== Answer ==
== Predict ==
18
21
34
8
5
== Answer ==
== Predict ==
29
21
4
20
32
== Answer ==
4
21
== Predict ==
18
23
27
22
12
== Answer ==
12
18
22
== Predict ==
20
37
34
16
23
== Answer ==
37
== Predict ==
8
9
34
36
12
== Answer ==
== Predict ==
18
27
23
35
32
== Answer ==
23
27
32
== Predict ==
15
39
14
23
33
== Answer ==
== Predict ==
29
19
4
18
37
== Answer ==
== Predict ==
20
4
9
29
26
== Answer ==
== Predict ==
34
18
8
38
11
== Answer ==
== Predict ==
35
34
15
22
24
== Answer ==
== Predict ==
26
33
34
32
12
== Answer ==
12
26
34
== Predict ==
35
32
23
12
18
== Answer ==
23
== Predict ==
5
23
21
19
8
== Answer ==
== Predict ==
21
36
5
15
19
== Answer ==
15
36
== Predict ==
3
30
37
39
25
== Answer ==
== Predict ==
18
5
36
12
9
== Answer ==
== Predict ==
19
4
29
36
23
== Answer ==
19
== Predict ==
29
37
21
19
4
== Answer ==
21
== Predict ==
29
18
21
26
9
== Answer ==
18
21
== Predict ==
37
21
23
19
4
== Answer ==
37
== Predict ==
18
28
12
8
38
== Ans

29
32
26
18
9
== Answer ==
9
== Predict ==
20
21
34
2
3
== Answer ==
2
== Predict ==
20
34
10
16
37
== Answer ==
== Predict ==
29
19
27
37
18
== Answer ==
18
27
== Predict ==
35
20
16
4
33
== Answer ==
20
== Predict ==
16
3
33
25
34
== Answer ==
33
== Predict ==
16
35
33
17
1
== Answer ==
16
== Predict ==
19
37
29
4
21
== Answer ==
29
37
== Predict ==
33
16
25
15
17
== Answer ==
17
== Predict ==
22
26
38
18
9
== Answer ==
18
== Predict ==
15
20
21
3
35
== Answer ==
15
20
== Predict ==
21
15
3
13
22
== Answer ==
15
== Predict ==
21
37
22
3
26
== Answer ==
== Predict ==
13
8
5
17
36
== Answer ==
== Predict ==
38
14
28
39
16
== Answer ==
38
== Predict ==
35
10
18
5
19
== Answer ==
10
== Predict ==
39
20
29
8
36
== Answer ==
== Predict ==
17
28
10
24
30
== Answer ==
28
== Predict ==
34
18
26
12
38
== Answer ==
38
== Predict ==
18
35
38
6
28
== Answer ==
== Predict ==
29
21
37
22
19
== Answer ==
37
== Predict ==
13
8
36
22
19
== Answer ==
== Predict ==
38
28
14
29
34
== Answer ==
== Predict

In [331]:
temp = df[-1:].values
temp = temp[:, 1:]
# temp
pred = model.predict(temp)

for idx, val in enumerate(pred):
    top_inds = val.argsort()[::-1][:5]
    print("== Predict ==")
    for i in top_inds:
        print(i+1)

== Predict ==
26
34
27
1
37
